In [62]:
import sys
import math
from sympy import symbol, sin, tan, log, diff, lambdify

# Нелинейные уравнения и системы

## Метод Ньютона

Реализуем Метод Ньютона (метод касательных):

In [2]:
def Newton(f: object, dfdx: object, x: float, eps: float=0.001) -> (float, int):
    f_value = f(x)
    iteration_counter = 0
    while abs(f_value) > eps and iteration_counter < 100:
        try :
            x = x - f_value/dfdx(x)
        except ZeroDivisionError as err:
            print(err)
            sys.exit(1)
        f_value = f(x)
        iteration_counter += 1
        
    if abs(f_value) > eps:
        iteration_counter=-1
    return x, iteration_counter

Функцию и ее производную реализуем средствами стандартной библиотеки math

In [3]:
def f1(x):
    return math.sin(x) - x

def f2(x):
    return math.tan(x) - x**2

def df1dx(x):
    return math.cos(x) - 1

Ожидаемое решение уравнения x = sin(x) ожидаемо находится в 0. Попробуем изменить начальное приближение, используя значения, достаточно близкие к 0. 

In [6]:
%%time

for x in [i/2 for i in range(-2, 3)]:
    print(Newton(f1, df1dx, x))

(-0.12780966756070827, 5)
(-0.14713338829825784, 3)
(0.0, 0)
(0.14713338829825784, 3)
(0.12780966756070827, 5)
Wall time: 1.01 ms


Так же имеет смысл воспользоваться символьными вычислениями. Несмотря на заметное снижение скорости работы, нет необходимости задумывать о самостоятельном аналитическом вычислении производной.

In [7]:
%%time

x = symbol.Symbol('x')
f_expr = sin(x) - x
dfdx_expr = diff(f_expr, x)

f_one = lambdify([x], f_expr)
dfdx_one = lambdify([x], dfdx_expr)

for x in [i/2 for i in range(-2, 3)]:
    print(Newton(f_one, dfdx_one, x))

(-0.12780966756070827, 5)
(-0.14713338829825784, 3)
(0.0, 0)
(0.14713338829825784, 3)
(0.12780966756070827, 5)
Wall time: 3.99 ms


Аналогично поступим и с уравнением x^2 = tg(x). Ожидаем решение в 0.

In [8]:
%%time

x = symbol.Symbol('x')
f_expr = x**2 - tan(x)
dfdx_expr = diff(f_expr, x)

f_two = lambdify([x], f_expr)
dfdx_two = lambdify([x], dfdx_expr)

for x in range(-5, 10):
    print(Newton(f_two, dfdx_two, x))

(-1.8538791463226192, 7)
(-1.8539085347956428, 6)
(-1.853927447916057, 5)
(-1.853876337883306, 3)
(-0.0005219408752792642, 4)
(0, 0)
(-1.4274290365031816e-06, 6)
(-2.648836078104986e-05, 11)
(-1.9613888469936433e-05, 7)
(-4.537253899664778e-05, 8)
(-0.0001404098530976665, 6)
(-0.0005602706420982546, 4)
(-0.0009487861590114152, 8)
(-0.00034129377588961243, 14)
(-6.907072372023745e-05, 6)
Wall time: 5.98 ms


## Метод хорд (метод секущих)

Далее реализуем метод хорд

In [9]:
def Secant(f: object, x0: float, eps: float=1e-7, kmax: int=1e3) -> (float, int):
    x, x_prev, i = x0, x0 + 2 * eps, 0
    
    while abs(x - x_prev) >= eps and i < kmax:
        x = x - f(x) / (f(x) - f(x_prev)) * (x - x_prev)
        x_prev = x
        i += 1
    return x, i

In [10]:
for x in range(-2, 3):
    print(Secant(f1, x))

(-1.2298096393445845, 1)
(-0.655145008902154, 1)
(0.0, 1)
(0.6551451352725557, 1)
(1.2298097381180813, 1)


In [11]:
for x in range(-2, 3):
    print(Secant(f2, x))

(-1.8143149602174096, 1)
(-0.5286334069064893, 1)
(0.0, 1)
(0.6089792869593829, 1)
(5.485714687408281, 1)


## Метод половинного деления (метод дихотомии)

In [54]:
def Bisection(f: object, a: float, b: float, eps: float=0.001) -> (float, int):
    i = 0
    while abs(b - a) > eps:
        dx = (b - a) / 2
        x = a + dx
        
        if math.copysign(1, f(a)) != math.copysign(1, f(x)):
            b = x
        else:
            a = x
        i += 1
    try:
        return x, i
    
    except UnboundLocalError:
        print("Eps is more then lenght of the interval")
        return None

Увеличивая параметр eps мы будем приближаться к 0.

In [60]:
for eps in [10**i for i in range(-10, 1)]:
    print(Bisection(f1, -2.0, 2.0, eps=eps))

(2.1478626877069473e-08, 36)
(2.1420419216156006e-08, 32)
(2.2351741790771484e-08, 29)
(5.960464477539063e-08, 26)
(9.5367431640625e-07, 22)
(7.62939453125e-06, 19)
(6.103515625e-05, 16)
(0.0009765625, 12)
(0.0078125, 9)
(0.0625, 6)
(1.0, 2)


Соответственно если eps будет больше чем длина интервала, на котором мы ищим корень, то метод не имеет смысла.

In [61]:
for eps in [10**i for i in range(-10, 2)]:
    print(Bisection(f2, -2.0, 2.0, eps=eps))

(-5.820766091346741e-11, 36)
(-9.313225746154785e-10, 32)
(-7.450580596923828e-09, 29)
(-5.960464477539063e-08, 26)
(-9.5367431640625e-07, 22)
(-7.62939453125e-06, 19)
(-6.103515625e-05, 16)
(-0.0009765625, 12)
(-0.0078125, 9)
(-0.0625, 6)
(-1.0, 2)
Eps is more then lenght of the interval
None


## ln(x) + x – 2 = 0.

Подготовим функцию. Решение находится в точке 1.557, можно выбрать одну точку в качестве начального приближения, например точку x = 1.

In [71]:
def func(x):
    return math.log(x) + x - 2

x = symbol.Symbol('x')
f_expr = log(x) + x - 2
dfdx_expr = diff(f_expr, x)

f = lambdify([x], f_expr)
dfdx = lambdify([x], dfdx_expr)

In [74]:
%%time
print(Newton(f, dfdx, 1))

(1.5567209351351015, 2)
Wall time: 0 ns


In [76]:
%%time
print(Secant(f, 1))

(1.5000000249800194, 1)
Wall time: 0 ns


Довольно низкая точность, можно попробвоать изменить начальные параметры

In [86]:
%%time
print(Secant(f, 2))

(1.5379018717602126, 1)
Wall time: 0 ns


In [87]:
%%time
print(Bisection(f, -2, 2))

(1.5576171875, 12)
Wall time: 0 ns


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys
D:\Programs\Anaconda\lib\site-packages\numpy\lib\scimath.py:274: RuntimeWarning: divide by zero encountered in log
  return nx.log(x)
